In [2]:
import numpy as np
import pandas as pd
import os

path = "C:\\Users\pault\OneDrive\Documenten\GitHub\input" # pauls path
os.chdir(path)

train = pd.read_csv('train.tsv', delimiter='\t', encoding='utf-8')
print("train size:", train.size)

test = pd.read_csv('test.tsv', delimiter='\t', encoding='utf-8')
print("test size:", test.size)

train size: 11860280
test size: 4853513


In [3]:
train_x = train.drop("price", 1)
train_y = train[["price"]] # prices of training data
train_x.head(5)

,train_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,0,Complete with certificate of authenticity
